In [11]:
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader, PdfFileReader
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
from langchain.schema.runnable import RunnablePassthrough
import openai
from llama_index.llms.ollama import Ollama


In [29]:
from llama_index.core import *


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/admin/opt/miniconda3/envs/chatbot/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/cz/mf6k7s456v95hf4y3g4lyfz40000gn/T/ipykernel_13137/3764686171.py", line 1, in <module>
    from llama_index.core import *
  File "/Users/admin/opt/miniconda3/envs/chatbot/lib/python3.10/site-packages/llama_index/core/__init__.py", line 16, in <module>
    from llama_index.core.indices import (
  File "/Users/admin/opt/miniconda3/envs/chatbot/lib/python3.10/site-packages/llama_index/core/indices/__init__.py", line 32, in <module>
    from llama_index.core.indices.loading import (
  File "/Users/admin/opt/miniconda3/envs/chatbot/lib/python3.10/site-packages/llama_index/core/indices/loading.py", line 6, in <module>
    from llama_index.core.indices.registry import INDEX_STRUCT_TYPE_TO_INDEX_CLASS
  File "/Users/admin/opt/miniconda3/envs/chatbot/l

In [2]:
from EmbeddingsPDF import EmbeddingsPDF
vectorstore = EmbeddingsPDF().get_chroma_embeddings()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
import torch

MODEL_NAME = "susnato/phi-2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, return_token_type_ids=False)
tokenizer.pad_token = tokenizer.eos_token

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=None  # Remove quantization for now, add it back later if needed
)



generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 300
generation_config.temperature = 0.7
generation_config.top_p = 0.95
generation_config.do_sample = True
# generation_config.repetition_penalty = 1.15


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from transformers import TextGenerationPipeline
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

# text_generation_pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, generation_config=generation_config)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

In [5]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(
    pipeline=pipeline,
)

In [6]:
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

template = """
[INST] <>
Act as an Insurance expert. Use the following information to answer the question at the end.
<>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [7]:
from IPython.display import Markdown
prompt = "Are there any restrictions on cover provided by Vitality?"

outputs = pipeline(
    prompt,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)
Markdown(outputs[0]["generated_text"])

In [ ]:
from IPython.display import Markdown, display
from tqdm.notebook import tqdm

query = "Are there any restrictions on cover provided by Vitality?"
with tqdm(total=1, desc="Processing queries") as pbar:
    result_ = qa_chain(query)
    result = result_["result"].strip()
    pbar.update(1)


display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Processing queries:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/admin/opt/miniconda3/envs/chatbot/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)


In [ ]:
user_question = "What is Vitality?"

In [ ]:
vectorstore_as_retriever = vectorstore.as_retriever()# Replace with your actual retriever

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# Create ConversationalRetrievalChain with Phi-2
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    # retriever=Phi2Retriever(model, tokenizer),
    memory=memory,
    retriever=vectorstore_as_retriever, verbose=True,
)


ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,    max_lengt=512, temperature=0.6, top_p=0.95, repetition_penalty=1.2)
local_llm=HuggingFacePipeline(pipeline=pipe)
pipe.model.config.pad_token_id=pipe.model.config.eos_token_id